In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch.nn as nn
import torch
from bayesian_nnet.data import IQDataModel

In [3]:
iqdata = IQDataModel()
iqdata.setup(stage="")
dl = iqdata.train_dataloader()
batch, _, _ = dl.__iter__().__next__()
batch = batch[:10, :, :]

In [8]:
"""
Layer #1
---------
conv 3, 64
ReLU
max pool / 2
"""
conv1 =\
    nn.Sequential(
        nn.Conv1d(in_channels=2,
                  out_channels=128,
                  kernel_size=3,
                  padding=1),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=3,
                     stride=2,
                     padding=1))

"""
Layer #2
---------
conv 3, 64
ReLU
max pool / 2
"""
conv2 =\
    nn.Sequential(
        nn.Conv1d(in_channels=64,
                  out_channels=64,
                  kernel_size=3,
                  padding=1),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=3,
                     stride=2,
                     padding=1))

"""
Layer #3
---------
conv 3, 128
ReLU
max pool / 2
"""
conv3 =\
    nn.Sequential(
        nn.Conv1d(in_channels=64,
                  out_channels=128,
                  kernel_size=3,
                  padding=1),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=3,
                     stride=2,
                     padding=1))

"""
Layer #4
---------
conv 3, 128
ReLU
max pool / 2
"""
conv4 =\
    nn.Sequential(
        nn.Conv1d(in_channels=128,
                  out_channels=128,
                  kernel_size=3,
                  padding=1),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=3,
                     stride=2,
                     padding=1))

lstm = nn.LSTM(128,
               64,
               num_layers=2,
               batch_first=True)

In [14]:
layer_out = conv1(batch)
print(layer_out.shape)
layer_out = conv2(layer_out)
print(layer_out.shape)
layer_out = conv3(layer_out)
print(layer_out.shape)
layer_out = conv4(layer_out)
print(layer_out.shape)
layer_out = torch.permute(layer_out, (0, 2, 1))
layer_out, _ = lstm(layer_out)
layer_out = torch.permute(layer_out, (0, 2, 1))
print(layer_out.shape)
layer_out = torch.flatten(layer_out, start_dim=1)
print(layer_out.shape)

torch.Size([10, 64, 1024])
torch.Size([10, 64, 512])
torch.Size([10, 128, 256])
torch.Size([10, 128, 128])
torch.Size([10, 64, 128])
torch.Size([10, 8192])


In [21]:
fc_layers =\
    nn.Sequential(nn.Linear(8192, 2048),
                  nn.Dropout(0.5),
                  nn.BatchNorm1d(2048),
                  nn.ReLU(),
                  nn.Linear(2048, 512),
                  nn.Dropout(0.5),
                  nn.BatchNorm1d(512),
                  nn.ReLU(),
                  nn.Linear(512, 18))

fc_layers(layer_out).shape

torch.Size([10, 18])

In [17]:
from bayesian_nnet.network import IQDataEmbedder, FrequentistClassifier

In [20]:
iq_embed = IQDataEmbedder()
mod_clf = FrequentistClassifier()
layer_out = iq_embed(batch)
print(layer_out.shape)
mod_clf(layer_out).shape

torch.Size([10, 8192])


torch.Size([10, 18])